# Homework

## Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

```bash
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
```

What's the version of dlt that you installed?

In [3]:
# Install dlt with Qdrant support and Qdrant client silently (suppress output with -q flag)
%pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

# Import modules after installation
import dlt
import qdrant_client

# Import dlt to verify installation and check the version programmatically
print(f"dlt version: {dlt.__version__}")  # Format output clearly for homework answer

# Also verify qdrant client installation
try:
    import pkg_resources
    qdrant_version = pkg_resources.get_distribution('qdrant-client').version
    print(f"qdrant-client version: {qdrant_version}")
    print("✅ All required packages installed successfully!")
except (ImportError, pkg_resources.DistributionNotFound):
    print("❌ Error: qdrant-client installation failed")

Note: you may need to restart the kernel to use updated packages.
dlt version: 1.13.0
qdrant-client version: 1.14.3
✅ All required packages installed successfully!
qdrant-client version: 1.14.3
✅ All required packages installed successfully!


/tmp/ipykernel_4213/3386569441.py:13: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## dlt Resourse

For reading the FAQ data, we have this helper function:

```python
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc
```

Annotate it with `@dlt.resource`. We will use it when creating
a dlt pipeline.

In [4]:
import dlt
import requests

@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

## Question 2. dlt pipeline

Now let's create a pipeline. 

We need to define a destination for that. Let's use the `qdrant` one:

```python
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)
```

In this case, we tell dlt (and Qdrant) to create a folder with
our data, and the name for it will be `db.qdrant`

Let's run it:

```python
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)
```

How many rows were inserted into the `zoomcamp_data` collection?

Look for `"Normalized data for the following tables:"` in the trace output.

In [5]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Run started at 2025-07-09 23:54:12.962410+00:00 and COMPLETED in 6.26 seconds with 4 steps.
Step extract COMPLETED in 0.54 seconds.

Load package 1752105254.6430333 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752105254.6430333 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 3.96 seconds.
Pipeline zoomcamp_pipeline load step completed in 3.95 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/workshops/dlt/db.qdrant location to store data
Load package 1752105254.6430333 is LOADED and contains no failed jobs

Step run COMPLETED in 6.26 seconds.
Pipeline zoomcamp_pipeline load step completed in 3.95 seconds
1 load package(s) were loaded to dest

## Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the `meta.json` file created
in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.

In [13]:
# Extract embedding model from the vectors configuration
if 'vectors' in collection_info:
    vectors_config = collection_info['vectors']
    for vector_name in vectors_config.keys():
        print(f"Embedding model: {vector_name}")


Embedding model: fast-bge-small-en
